# **_Data Engineering Basics_**

This notebook has a **didactic** purpose, regarding the practical application of a simple **data engineering** process using **postgreSQL** and **Python**.

## **Table of Contents**

1. [Database Connection](#1-Database-Connection)
2. [Load Data](#2-Load-Data)
3. [Modeling Data](#3-Modeling_Data)
4. [Verifiyng Data](#4-Verifiyng_Data)

## **Packages used in this notebook**

In [ ]:
# Packages used in this notebook
import psycopg2
from psycopg2.extensions import cursor, connection
import pandas as pd
from pandas import DataFrame

## **1. Database Connection**

In [ ]:
# Usually stored in environment variables for security and portability reasons
db_configs = {
    "user": "...",
    "password": "...",
    "dbname": "...",
    "host": "..."
}

In [ ]:
def connect_with_database(**db_config: dict) -> tuple[connection, cursor]:
    """Connect with PostgreSQL database

    :param db_config: Dictionary or key arguments with database configurations
    :returns: A tuple with database connection and cursor 
    """

    try:
        conn = psycopg2.connect(**db_config)
        conn.set_session(autocommit=True)
        cur = conn.cursor()

        return conn, cur

    except psycopg2.Error as e:
        print(
            F"Could not establish connection to postgres database.\n{e}"
        )

In [ ]:
# Connecting to the default database
conn, cur = connect_with_database(dbname="...", host="...",
                                  password="...", user="...")

In [ ]:
# Creating sparkify database with UTF8 enconding
cur.execute("DROP DATABASE ...")
cur.execute("CREATE DATABASE ...")

# Commiting and closing the last connection
conn.commit()
conn.close()

In [ ]:
# Connecting to the new sparkify database
conn, cur = connect_with_database(**db_configs)

## **2. Load data** 

In [ ]:
accounts_data = pd.read_csv("data/Wealth-AccountData.csv")
accounts_data.head(3)

In [ ]:
accounts_country = pd.read_csv("data/Wealth-AccountsCountry.csv")
accounts_country.head(3)

In [ ]:
account_series = pd.read_csv("data/Wealth-AccountSeries.csv")
account_series.head(3)

#### **2.1 Filtering data**

In [ ]:
# Columns that will be filtered from dataframes

accounts_data_selected_columns = ["Country Name", "Country Code", "Series Name", "Series Code", 
                                  "1995 [YR1995]", "2000 [YR2000]", "2005 [YR2005]", "2010 [YR2010]"]

accounts_country_selected_columns = ["Code", "Short Name", "Table Name", "Long Name", "Currency Unit"]

accounts_series_selected_columns = ["Code", "Topic", "Indicator Name", "Long definition"]

In [ ]:
accounts_country_filtered_columns = accounts_country.loc[0:12, accounts_country_selected_columns]
accounts_country_filtered_columns.head(3)

In [ ]:
accounts_series_filtered_columns = account_series.loc[0:12, accounts_series_selected_columns]
accounts_series_filtered_columns.head(3)

In [ ]:
accounts_data_filtered_columns = accounts_data.loc[0:12, accounts_data_selected_columns]
accounts_data_filtered_columns.head(3)

## **3. Modeling data**

In [ ]:
def create_table(table_name: str, table_columns: str) -> None:
    """Create a table
    
    :param table_name: Name of the table to be created
    :param table_columns: Column names belonging to this table
    
    :returns: None
    """
    
    cur.execute(F"""
                 CREATE TABLE IF NOT EXISTS {table_name} ({table_columns})
                 """)
    
    conn.commit()

In [ ]:
accountscountry_columns_with_type_definition = """
                                               country_code VARCHAR PRIMARY KEY,
                                               short_name VARCHAR,
                                               table_name VARCHAR,
                                               long_name VARCHAR,
                                               currency_unit VARCHAR
                                               """

In [ ]:
create_table(table_name="accountscountry",
             table_columns=accountscountry_columns_with_type_definition)

In [ ]:
accountsdata_columns_with_type_definition = """
                                            country_name VARCHAR,
                                            country_code VARCHAR,
                                            indicator_name VARCHAR,
                                            indicator_code VARCHAR,
                                            year_1995 numeric,
                                            year_2000 numeric,
                                            year_2005 numeric,
                                            year_2010 numeric
                                            """

In [ ]:
create_table(table_name="accountsdata",
             table_columns=accountsdata_columns_with_type_definition)

In [ ]:
accountsseries_columns_with_type_definition = """
                                              series_code VARCHAR,
                                              topic VARCHAR,
                                              indicator_name VARCHAR,
                                              short_definition VARCHAR
                                              """

In [ ]:
create_table(table_name="accountsseries",
             table_columns=accountsseries_columns_with_type_definition)

#### **3.1 Inserting data** 

In [ ]:
def insert_in_table(table_name: str, columns: str,  df: DataFrame) -> None:
    """Insert Dataframe data into a table
    
    :param table_name: Name of the table where the data will be inserted
    :param columns: Columns of the table where the data will be inserted
    :param df: Dataframe where data will be imported from
    
    :returns: None
    """
    
    for _, row in df.iterrows():
        cur.execute(F"""
                     INSERT INTO {table_name} ({columns})
                     VALUES {tuple(row)};
                     """)

In [ ]:
accountscountry_columns = accountscountry_columns_with_type_definition.replace(" VARCHAR", "") \
                                                                      .replace(" PRIMARY KEY", "")

insert_in_table(table_name="accountscountry",
                columns=accountscountry_columns,
                df=accounts_country_filtered_columns)

In [ ]:
accountsdata_columns = accountsdata_columns_with_type_definition.replace(" VARCHAR", "") \
                                                                .replace(" numeric", "")

insert_in_table(table_name="accountsdata",
                columns=accountsdata_columns,
                df=accounts_data_filtered_columns)

In [ ]:
accountsseries_columns = accountsseries_columns_with_type_definition.replace(" VARCHAR", "")


insert_in_table(table_name="accountsseries",
                columns=accountsseries_columns,
                df=accounts_series_filtered_columns)

## **4. Verifiyng data**

In [36]:
for table_name in ("AccountsCountry", "AccountsData", "AccountsSeries"):
    cur.execute(F"SELECT * FROM {table_name} LIMIT 5;")
    rows = cur.fetchall()
    
    print(table_name, *rows, sep="\n")

AccountsCountry
('ALB', 'Albania', 'Albania', 'Republic of Albania', 'Albanian lek')
('ARG', 'Argentina', 'Argentina', 'Argentine Republic', 'Argentine peso')
('ARM', 'Armenia', 'Armenia', 'Republic of Armenia', 'Armenian dram')
('AUS', 'Australia', 'Australia', 'Commonwealth of Australia', 'Australian dollar')
('AUT', 'Austria', 'Austria', 'Republic of Austria', 'Euro')
AccountsData
('Albania', 'ALB', 'Human capital (constant 2018 US$)', 'NW.HCA.TO', Decimal('44900000000'), Decimal('43600000000'), Decimal('54400000000.0'), Decimal('68100000000.0'))
('Albania', 'ALB', 'Human capital per capita (constant 2018 US$)', 'NW.HCA.PC', Decimal('14072.79'), Decimal('14118.13'), Decimal('18072.3'), Decimal('23362.86'))
('Albania', 'ALB', 'Human capital per capita, employed female (constant 2018 US$)', 'NW.HCA.FEMP.PC', Decimal('3468.858'), Decimal('3424.007'), Decimal('2222.115'), Decimal('2889.15'))
('Albania', 'ALB', 'Human capital per capita, employed male (constant 2018 US$)', 'NW.HCA.MEMP.P